In [25]:
import os
import json
import matplotlib.pyplot as plt

# Directory paths
directory_path_test_cfeddc = '/proj/sourasb-220503/TCSS-DIPA2/Fed-DIPA2/results/client_level/CFedDC_rl1_C2/local_test'
directory_path_test_siloed = '/proj/sourasb-220503/TCSS-DIPA2/Fed-DIPA2/results/client_level/siloed/local_test'
output_dir = './user_plots'  # Folder to save individual plots
os.makedirs(output_dir, exist_ok=True)

# File lists
cfeddc_files = sorted([f for f in os.listdir(directory_path_test_cfeddc) if f.endswith('.json')])
siloed_files = sorted([f for f in os.listdir(directory_path_test_siloed) if f.endswith('.json')])

for file_name in cfeddc_files:
    if file_name in siloed_files:
        cfeddc_path = os.path.join(directory_path_test_cfeddc, file_name)
        siloed_path = os.path.join(directory_path_test_siloed, file_name)

        try:
            with open(cfeddc_path, 'r') as f1, open(siloed_path, 'r') as f2:
                data_cfeddc = json.load(f1)
                data_siloed = json.load(f2)

                user_id = data_siloed.get("user_id", file_name.split("_")[1])
                resource_category = data_cfeddc.get("resource_category", "Unknown")

                mae_cfeddc = data_cfeddc.get("validation_results", {}).get("info_cmae", [])
                mae_siloed = data_siloed.get("validation_results", {}).get("info_cmae", [])

                if mae_cfeddc and mae_siloed:
                    mae_cfeddc_val = min(mae_cfeddc)
                    mae_siloed_val = mae_siloed[-1]

                    # Plot
                    fig, ax = plt.subplots(figsize=(4, 5))
                    ax.bar(['CFedDC', 'Siloed'], [mae_cfeddc_val, mae_siloed_val], color=['gray', 'black'])
                    ax.set_title(f'User {user_id}\n({resource_category})', fontsize=18)
                    ax.set_ylabel('info_cmae', fontsize=16)
                    ax.set_ylim(0, max(mae_cfeddc_val, mae_siloed_val) * 1.2)
                    ax.grid(True, axis='y')

                    # Save plot
                    plt.tight_layout()
                    filename = f'user_{user_id}_{resource_category}.png'.replace(' ', '_')
                    plt.savefig(os.path.join(output_dir, filename))
                    plt.close()

        except Exception as e:
            print(f"Error processing {file_name}: {e}")


/tmp/ipykernel_4012939/2877756682.py:40: UserWarning: Attempting to set identical low and high ylims makes transformation singular; automatically expanding.
  ax.set_ylim(0, max(mae_cfeddc_val, mae_siloed_val) * 1.2)
